# LangChain Runnable 컴포넌트

이 노트북에서는 LangChain의 다양한 Runnable 컴포넌트들을 살펴보겠습니다:

- [기본 체인 구성](#기본-체인-구성)
- [RunnablePassthrough](#runnablepassthrough): 입력을 그대로 전달하거나 키를 추가
- [RunnableLambda](#runnablelambda): 정의된 함수를 체인에 통합
- [RunnableParallel](#runnableparallel): 여러 체인을 병렬로 실행
- [itemgetter](#itemgetter): 딕셔너리 값을 쉽게 접근

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## 기본 체인 구성

LangChain에서는 파이프라인(`|`) 연산자를 사용하여 체인을 쉽게 구성할 수 있습니다.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


# prompt와 llm을 생성
prompt = PromptTemplate.from_template("{num}의 제곱은?")
llm = ChatOpenAI(temperature=0)

# 파이프라인 연산자로 체인 구성
chain = prompt | llm

In [ ]:
# 딕셔너리로 입력값 전달
chain.invoke({"num": 5})

In [ ]:
# 값이 하나일 경우 딕셔너리가 아니어도 작동
chain.invoke(5)

## RunnablePassthrough

`RunnablePassthrough`는 입력을 변경하지 않거나 추가 키를 더하여 전달할 수 있습니다.

- `RunnablePassthrough()`가 단독으로 호출되면, 단순히 입력을 받아 그대로 전달합니다.
- `RunnablePassthrough.assign(...)`으로 호출되면, 입력을 받아 assign 함수에 전달된 추가 인수를 추가합니다.

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# 입력을 그대로 전달
RunnablePassthrough().invoke({"num": 10})

In [ ]:
# 사용예시: RunnablePassthrough를 사용한 체인 구성
runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

runnable_chain.invoke(10)

In [ ]:
# assign을 사용하는 경우 새로운 키-값 쌍이 할당되어 합쳐짐
(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})

## RunnableLambda

`RunnableLambda`는 정의된 함수를 프롬프트에 통합할 수 있게 해줍니다. 이를 통해 동적인 값을 체인 내에서 생성할 수 있습니다.

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime

# 오늘 날짜를 반환하는 함수
def get_today(a):
    print(f"입력받은 매개변수의값: {a}")
    return datetime.today().strftime("%b-%d")

# 오늘 날짜 출력
get_today(None)

In [ ]:
# RunnableLambda를 사용한 체인 구성
prompt = PromptTemplate.from_template(
    "{today}가 생일인 유명인 {n}명을 나열하세요. 생년월일을 표기해 주세요."
)
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

chain = (
    {"today": RunnableLambda(get_today), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 출력
print(chain.invoke(3))

In [ ]:
# RunnableLambda에 전달되는 값의 활용 예시
def get_number(a):
    print(f"입력받은 매개변수의값: {a}")
    return a ** a

prompt = PromptTemplate.from_template(
    "{number}의 소인수분해에는 {n}이 들어가있나요? 소인수분해시 전체 수를 보여주세요"
)

chain = (
    {"number": RunnableLambda(get_number), "n": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print(chain.invoke(3))

## RunnableParallel

`RunnableParallel`은 여러 체인을 병렬로 실행할 수 있게 해주는 클래스입니다. 각 체인이 독립적으로 동작하므로 여러 작업을 동시에 수행할 때 유용합니다.

In [ ]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스 생성
runnable = RunnableParallel(
    # 입력을 그대로 전달
    passed=RunnablePassthrough(),
    # 'mult' 키에 lambda 함수 결과를 할당
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 'modified' 키에 lambda 함수 결과를 할당
    modified=lambda x: x["num"] + 1,
)

# 실행
runnable.invoke({"num": 1})

In [ ]:
# 실제 사용 예시
chain1 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 수도는?")
    | ChatOpenAI()
)
chain2 = (
    {"country": RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 면적은?")
    | ChatOpenAI()
)

# 병렬로 실행하는 체인 구성
combined_chain = RunnableParallel(capital=chain1, area=chain2)
combined_chain.invoke("대한민국")

## itemgetter

`itemgetter`는 딕셔너리 값에 쉽게 접근할 수 있는 방법을 제공합니다. 여러 키 값을 한 번에 받아올 수 있습니다.

In [ ]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


# 문장의 길이를 반환하는 함수
def length_function(text):
    return len(text)


# 두 문장의 길이를 곱한 값을 반환하는 함수
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환하는 함수
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])


prompt = ChatPromptTemplate.from_template("{a} + {b}는 무엇인가요?")
model = ChatOpenAI()

# itemgetter를 사용한 체인 구성
chain = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
)

# 실행
chain.invoke({"word1": "hello", "word2": "world"})